In [1]:
%pip install --upgrade pip
%pip install numpy pandas
%pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


# 신경망 (Neural Networks)

신경망은 `torch.nn` 패키지를 사용하여 생성할 수 있다.

`nn`은 모델을 정의하고 미분하는데 `autograd`를 사용하며, `nn.Module`은 'layer'와 `ouput`을 반환하는 'forward(input)' 메서드를 포함한다.

### 신경망의 일반적인 학습 과정

- 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의한다.
- 데이터셋(dataset) 입력을 반복한다.
- 입력을 신경망에서 전파(process)한다.
- 손실(loss)을 계산한다.
- 변화도(gradient)를 신경망의 매개변수들에 역으로 전파한다.
- 신경망의 가중치를 갱신한다.
    - `weight = weight - learning_rate * gradient`

### 요약

- `torch.Tensor`
    - `backward()` 같은 autograd 연산을 지원하는 다차원 배열이다.
    - tensor에 대한 변화도를 갖는다.
- `nn.Module`
    - 신경망 모듈이다.
    - 매개변수를 캡슐화(encapsulation)하는 간편한 방법으로, GPU 이동, 내보내기(exporting), 불러오기(loading) 등의 작업을 위한 헬퍼(helper)를 제공한다.
- `nn.Parameter`
    - Tensor의 한 종류로, `Module`에 속성으로 할당될 때 자동으로 매개변수로 등록된다.
- `autograd.Function`
    - autograd 연산의 순방향과 역방향 정의를 구현한다.
    - 모든 Tensor 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Tensor를 생성하고 이력(history)을 인코딩하는 함수들과 연결하고 있다.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### 신경망 정의하기

`forward` 함수만 정의하고 나면, `backward` 함수는 `autograd`를 사용하여 자동으로 정의된다. `forward` 함수에서는 어떠한 Tensor 연산을 사용해도 된다.

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
###
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


모델의 학습 가능한 매개변수들은 `net.parameters()`에 의해 반환된다.

In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [7]:
out = net(torch.randn(1, 1, 32, 32))
print(out)

tensor([[-0.0814, -0.0515,  0.1066,  0.1615,  0.0440,  0.0045,  0.1596,  0.1507,
         -0.0811,  0.0235]], grad_fn=<AddmmBackward0>)


모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고, 무작위 값으로 역전파 한다.

In [8]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## 손실 함수(Loss Function)

손실 함수는 (output, target)을 한 쌍의 입력으로 받아, 출력이 정답으로부터 얼마나 멀리 떨어져 있는지 추정하는 값을 계산한다.

`nn` 패키지에는 여러가지 손실 함수들이 존재한다.
간단한 손실 함수로는 출려과 대상간의 평균제곱오차(MSE, mean squared error)를 계산하는 `nn.MSEloss`가 있다.

In [10]:
output = net(torch.randn(1, 1, 32, 32))
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.8203, grad_fn=<MseLossBackward0>)


`loss.backward()`를 실행할 때, 전체 그래프는 신경망의 매개변수에 대해 미분되며, 그래프 내의 `required_grad=True`인 모든 Tensor는 변화도가 누적된 `.grad` Tensor를 갖게 된다.

In [11]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## 역전파 (Backprop)

오차를 역전파하기 위해서는 `loss.backward()` 해주면 된다.
기존에 계산된 변화도의 값을 누적시키고 싶지 않다면 기존에 계산된 변화도를 0으로 만다는 작업이 필요하다.

In [12]:
net.zero_grad()

print("conv1.bias.grad before backward:")
print(net.conv1.bias.grad)
print()

loss.backward()

print("conv1.bias.grad after backward:")
print(net.conv1.bias.grad)

conv1.bias.grad before backward:
None

conv1.bias.grad after backward:
tensor([ 0.0182,  0.0032, -0.0100, -0.0002,  0.0162,  0.0240])


## 가중치 갱신

실제로 많이 사용되는 가장 단순한 갱신 규칙은 확률적 경사하강법(SGD, Stochastic Gradient Descent)이다.

```python
weight = weight - learning_rate * gradient
```

In [13]:
learning_rate = 1e-2
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

신경망을 구성할 때 SGD, Nesterov-SGD, Adam, RMSProp 등과 같은 다양한 갱신 규칙을 사용하고 싶다면,
`torch.optim`라는 패키지를 사용하면 된다.

In [14]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(torch.randn(1, 1, 32, 32))
loss = criterion(output, target)
loss.backward()
optimizer.step()